In [2]:
# prompt: mount

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# ==========================================
# ✅ Tea Leaf Augmentation Script (Fixed)
# ==========================================
import os
import random
import time
import numpy as np
import cv2
from PIL import Image, ImageEnhance

# --- Google Drive Path Configuration ---
source_folder = r"/content/drive/MyDrive/Tea Augmentation/Tea Algal Spot"
target_folder = r"/content/drive/MyDrive/Tea Augmentation/Tea Algal Spotoutput"
target_images = 1000

# --- Ensure Target Folder Exists ---
os.makedirs(target_folder, exist_ok=True)

# --- PIL-Based Augmentation ---
def augment_image_pil(image, count):
    brightness_factor = random.uniform(0.7, 1.3)
    contrast_factor = random.uniform(0.7, 1.3)
    color_factor = random.uniform(0.7, 1.3)

    enhanced_image = ImageEnhance.Color(
        ImageEnhance.Contrast(
            ImageEnhance.Brightness(image).enhance(brightness_factor)
        ).enhance(contrast_factor)
    ).enhance(color_factor)

    filename = f"aug_pil_{count}_b{brightness_factor:.2f}_c{contrast_factor:.2f}_col{color_factor:.2f}.jpg"
    return enhanced_image, filename

# --- OpenCV-Based Augmentation ---
def augment_image_cv2(image_path, count):
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Could not read image: {image_path}")

    height, width = image.shape[:2]
    augmented_images = []

    # Center Crops
    for crop_ratio in [0.8, 0.6]:
        crop_h, crop_w = int(height * crop_ratio), int(width * crop_ratio)
        start_x, start_y = (width - crop_w) // 2, (height - crop_h) // 2
        cropped = image[start_y:start_y + crop_h, start_x:start_x + crop_w]
        resized = cv2.resize(cropped, (width, height), interpolation=cv2.INTER_AREA)
        filename = f"cv2_crop_{crop_ratio}_{count}.jpg"
        augmented_images.append((resized, filename))

    # Center Scaling
    for scale in [1.1, 1.3]:
        scaled_h, scaled_w = int(height * scale), int(width * scale)
        scaled = cv2.resize(image, (scaled_w, scaled_h), interpolation=cv2.INTER_LINEAR)
        crop_y = (scaled_h - height) // 2
        crop_x = (scaled_w - width) // 2
        cropped = scaled[crop_y:crop_y + height, crop_x:crop_x + width]
        filename = f"cv2_scale_{scale}_{count}.jpg"
        augmented_images.append((cropped, filename))

    # Flip
    augmented_images.append((cv2.flip(image, 1), f"cv2_flip_h_{count}.jpg"))
    augmented_images.append((cv2.flip(image, 0), f"cv2_flip_v_{count}.jpg"))

    # Rotation
    for angle in [15, -15, 30, -30]:
        M = cv2.getRotationMatrix2D((width // 2, height // 2), angle, 1)
        rotated = cv2.warpAffine(image, M, (width, height), flags=cv2.INTER_LINEAR)
        filename = f"cv2_rotate_{angle}_{count}.jpg"
        augmented_images.append((rotated, filename))

    # Shifting
    for dx, dy in [(10, 0), (-10, 0), (0, 10), (0, -10)]:
        M = np.float32([[1, 0, dx], [0, 1, dy]])
        shifted = cv2.warpAffine(image, M, (width, height), flags=cv2.INTER_LINEAR)
        filename = f"cv2_shift_{dx}_{dy}_{count}.jpg"
        augmented_images.append((shifted, filename))

    # Zoom (random zoom in/out between 0.8 to 1.2)
    for zoom_factor in [0.8, 1.0, 1.2]:
        if zoom_factor == 1.0:
            zoomed = image.copy()
        elif zoom_factor > 1.0:
            zh, zw = int(height * zoom_factor), int(width * zoom_factor)
            zoomed_large = cv2.resize(image, (zw, zh), interpolation=cv2.INTER_LINEAR)
            start_x, start_y = (zw - width) // 2, (zh - height) // 2
            zoomed = zoomed_large[start_y:start_y + height, start_x:start_x + width]
        else:
            zh, zw = int(height * zoom_factor), int(width * zoom_factor)
            zoomed_small = cv2.resize(image, (zw, zh), interpolation=cv2.INTER_LINEAR)
            zoomed = np.zeros_like(image)
            start_x, start_y = (width - zw) // 2, (height - zh) // 2
            zoomed[start_y:start_y + zh, start_x:start_x + zw] = zoomed_small

        filename = f"cv2_zoom_{zoom_factor}_{count}.jpg"
        augmented_images.append((zoomed, filename))

    # Noisy
    mean, stddev = 0, 10
    noise = np.random.normal(mean, stddev, image.shape).astype(np.float32)
    noisy_image = image.astype(np.float32) + noise
    noisy_image = np.clip(noisy_image, 0, 255).astype(np.uint8)
    filename = f"cv2_noisy_{count}.jpg"
    augmented_images.append((noisy_image, filename))

    # Brightness
    for brightness_factor in [0.7, 1.3]:
        bright_img = cv2.convertScaleAbs(image, alpha=brightness_factor, beta=0)
        filename = f"cv2_brightness_{brightness_factor}_{count}.jpg"
        augmented_images.append((bright_img, filename))

    return augmented_images

# --- Manage Max Number of Images ---
def manage_target_folder(folder, max_images):
    images = sorted(
        [f for f in os.listdir(folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))],
        key=lambda x: os.path.getctime(os.path.join(folder, x))
    )
    if len(images) > max_images:
        for img in images[max_images:]:
            os.remove(os.path.join(folder, img))

# --- Main Process ---
def main():
    print("🔍 Checking source folder...")
    if not os.path.isdir(source_folder):
        print(f"❌ Source folder not found: {source_folder}")
        return
    else:
        print(f"✅ Source folder found: {source_folder}")

    # List files
    files_in_source = os.listdir(source_folder)
    print(f"📂 Files found: {len(files_in_source)}")
    if files_in_source:
        print("🔸 First few files:", files_in_source[:5])

    image_paths = [
        os.path.join(source_folder, f)
        for f in files_in_source
        if f.lower().endswith(('.jpg', '.jpeg', '.png'))
    ]

    if not image_paths:
        print("⚠️ No valid image files (.jpg/.jpeg/.png) found in source folder.")
        return
    else:
        print(f"✅ Found {len(image_paths)} valid image(s) for augmentation.")

    random.shuffle(image_paths)

    start_time = time.time()
    current_count = len(os.listdir(target_folder))
    print(f"🚀 Starting augmentation. Already in target: {current_count} images.")

    while current_count < target_images:
        for img_path in image_paths:
            if current_count >= target_images:
                break
            try:
                with Image.open(img_path) as pil_image:
                    if pil_image.mode != "RGB":
                        pil_image = pil_image.convert("RGB")

                    pil_aug, pil_filename = augment_image_pil(pil_image, current_count)
                    pil_aug.save(os.path.join(target_folder, pil_filename), format="JPEG")
                    print(f"✅ Saved PIL image: {pil_filename}")
                    current_count += 1

                cv2_augments = augment_image_cv2(img_path, current_count)
                for aug_img, filename in cv2_augments:
                    if current_count >= target_images:
                        break
                    cv2.imwrite(os.path.join(target_folder, filename), aug_img)
                    print(f"✅ Saved CV2 image: {filename}")
                    current_count += 1

            except Exception as e:
                print(f"⚠️ Error processing {img_path}: {e}")

    manage_target_folder(target_folder, target_images)
    total = len(os.listdir(target_folder))
    print(f"\n✅ Augmentation complete. Total images: {total}")
    print(f"⏱ Time taken: {time.time() - start_time:.2f} seconds")

# --- Run ---
if __name__ == "__main__":
    main()


🔍 Checking source folder...
✅ Source folder found: /content/drive/MyDrive/Tea Augmentation/Tea Algal Spot
📂 Files found: 54
🔸 First few files: ['71.png', '_DSC7791.png', '_DSC7795.png', '_DSC7785.png', '65.png']
✅ Found 54 valid image(s) for augmentation.
🚀 Starting augmentation. Already in target: 0 images.
✅ Saved PIL image: aug_pil_0_b1.17_c0.88_col0.86.jpg
✅ Saved CV2 image: cv2_crop_0.8_1.jpg
✅ Saved CV2 image: cv2_crop_0.6_1.jpg
✅ Saved CV2 image: cv2_scale_1.1_1.jpg
✅ Saved CV2 image: cv2_scale_1.3_1.jpg
✅ Saved CV2 image: cv2_flip_h_1.jpg
✅ Saved CV2 image: cv2_flip_v_1.jpg
✅ Saved CV2 image: cv2_rotate_15_1.jpg
✅ Saved CV2 image: cv2_rotate_-15_1.jpg
✅ Saved CV2 image: cv2_rotate_30_1.jpg
✅ Saved CV2 image: cv2_rotate_-30_1.jpg
✅ Saved CV2 image: cv2_shift_10_0_1.jpg
✅ Saved CV2 image: cv2_shift_-10_0_1.jpg
✅ Saved CV2 image: cv2_shift_0_10_1.jpg
✅ Saved CV2 image: cv2_shift_0_-10_1.jpg
✅ Saved CV2 image: cv2_zoom_0.8_1.jpg
✅ Saved CV2 image: cv2_zoom_1.0_1.jpg
✅ Saved CV2 ima

In [6]:
import os

target_folder = r"/content/drive/MyDrive/Tea Augmentation/Output"
expected_count = 500

# Count .jpg images in the folder
image_count = len([
    f for f in os.listdir(target_folder)
    if f.lower().endswith('.jpg')
])

print(f"✅ Found {image_count} .jpg images in: {target_folder}")

if image_count == expected_count:
    print("🎉 Folder has exactly 500 images.")
elif image_count < expected_count:
    print(f"⚠️ Folder has fewer images than expected ({expected_count - image_count} missing).")
else:
    print(f"⚠️ Folder has more images than expected ({image_count - expected_count} extra).")


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Tea Augmentation/Output'

In [ ]:
import os

# Path to the main dataset folder containing 10 class subfolders
main_dataset_folder = r"/content/drive/MyDrive/Tea Augmentation"  # Change this as needed

# List of 10 class names (folder names inside main folder)
class_names = [
    "Citruspot", "Early_Mild_Spotting", "Fungal", "Healthy", "Healthy_Mildly",
    "Healthy_Yellowing", "Mild_Edge__Damage", "Senescent", "Slightly_Diseased", "Wrinkled_Leaf"
]

print("📊 Image count per class:\n")

# Loop through each class folder and count .jpg images
for class_name in class_names:
    class_path = os.path.join(main_dataset_folder, class_name)

    if not os.path.isdir(class_path):
        print(f"❌ {class_name} - Folder not found: {class_path}")
        continue

    jpg_count = len([
        f for f in os.listdir(class_path)
        if f.lower().endswith('.jpg')
    ])

    print(f"✅ {class_name}: {jpg_count} .jpg images")


📊 Image count per class:

❌ Citruspot - Folder not found: /content/drive/MyDrive/Hi/Citruspot
✅ Early_Mild_Spotting: 500 .jpg images
✅ Fungal: 500 .jpg images
✅ Healthy: 500 .jpg images
✅ Healthy_Mildly: 500 .jpg images
✅ Healthy_Yellowing: 500 .jpg images
✅ Mild_Edge__Damage: 500 .jpg images
✅ Senescent: 500 .jpg images
✅ Slightly_Diseased: 500 .jpg images
✅ Wrinkled_Leaf: 500 .jpg images
